In [8]:
import pandas as pd
import numpy as np

## Reading file 
results = pd.read_csv('submission.csv')
results.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.036132
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.030219
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.022902
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.676110
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.632976


In [9]:
results['label'] = np.where(results['prediction'] > 0.5, 1, 0)
results.head()

,customer_ID,prediction,label
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.036132,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.030219,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.022902,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.676110,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.632976,1


In [10]:
results['label'].value_counts() / results.shape[0]

0    0.797456
1    0.202544
Name: label, dtype: float64

In [7]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# import optuna
# from xgboost import XGBClassifier
from Amex_Metric import amex_metric

import os
import sagemaker

sess = sagemaker.Session()

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'AmericanExpress/Delinquency_Features_Filled.csv'
file_key_2 = 'AmericanExpress/test_delinquency_features.csv'
file_key_3 = 'AmericanExpress/amex_train_payment_spend_final.csv'
file_key_4 = 'AmericanExpress/amex_test_payment_spend_final.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
oscar_data = pd.read_csv(file_content_stream_1, usecols = ['D_44_median', 'D_44_mean', 'D_44_max', 
                                                           'D_75_max', 'D_75_mean', 'D_78_max', 
                                                           'D_78_mean', 'D_78_range', 'D_44_std', 
                                                           'D_75_median', 'D_78_std', 'D_74_mean',
                                                           'D_44_range', 'D_44_min', 'D_84_mean', 
                                                           'D_74_max', 'D_41_range', 'D_75_min', 
                                                           'D_44_IQR', 'D_84_range', 'target', 
                                                           'customer_ID'])

oscar_test = pd.read_csv(file_content_stream_2)

evan_data = pd.read_csv(file_content_stream_3)
evan_data = evan_data.drop(columns = ['target'], axis = 1)

evan_test = pd.read_csv(file_content_stream_4)

## joining files 
data = pd.merge(oscar_data, evan_data, on = 'customer_ID', how = 'left')

In [8]:
oscar_test.head()

,D_44_median,D_44_mean,D_44_max,D_75_max,D_75_mean,D_78_max,D_78_mean,D_78_range,D_44_std,D_75_median,...,D_74_mean,D_44_range,D_44_min,D_84_mean,D_74_max,D_41_range,D_75_min,D_44_IQR,D_84_range,customer_ID
0,0.007584,0.00708,0.009960,0.21000,0.204100,0.007587,0.004128,0.006081,0.002674,0.203500,...,0.219200,0.009026,0.000928,0.004710,0.22350,0.009804,0.201200,0.001915,0.009834,00000469ba478561f23a92a868bd366de6f6527a684c9a...
1,0.003616,0.00445,0.009926,0.00948,0.004864,0.009760,0.004940,0.009163,0.003029,0.004974,...,0.004456,0.009758,0.000165,0.005833,0.00877,0.009247,0.000130,0.005089,0.009544,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...
2,0.006062,0.00523,0.008360,0.07587,0.046100,0.009964,0.006954,0.008789,0.002171,0.068240,...,0.049470,0.007797,0.000566,0.005516,0.08040,0.007736,0.001328,0.002205,0.009239,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...
3,0.378906,0.38010,0.507300,0.74100,0.674000,0.009790,0.005880,0.008942,0.100403,0.676300,...,0.577000,0.251709,0.255600,0.003878,0.65200,0.162354,0.543000,0.241211,0.007572,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...
4,0.257812,0.24670,0.382000,0.40840,0.337200,1.003000,0.195800,1.002930,0.079651,0.335400,...,0.148000,0.256836,0.125200,0.004790,0.21720,0.008881,0.269300,0.006836,0.007835,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...


In [12]:
oscar_test.shape

(924621, 21)

In [9]:
evan_test.head()

,customer_ID,P_2_mean,P_2_median,P_2_sum,P_2_correlation,P_3_mean,S_25_mean,S_25_sum,S_25_std,S_25_mad,S_25_data_range,S_25_iqr
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.6016,0.5970,5.414,-0.484413,0.5737,0.9740,8.766,0.003436,0.003038,0.00879,0.005859
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.8623,0.8610,11.210,-0.459726,0.5530,0.9740,12.664,0.003073,0.002554,0.00830,0.005371
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.7490,0.7437,9.734,-0.398301,0.6710,0.9730,12.650,0.003055,0.002628,0.00830,0.005371
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.4746,0.4740,6.170,0.511864,0.6110,0.9730,12.650,0.003233,0.002855,0.00879,0.005859
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.3240,0.3162,4.215,-0.495118,0.6350,0.5396,7.016,0.363472,0.334700,0.81640,0.712158


In [11]:
evan_test.shape

(589833, 12)